In [ ]:
#default_exp filter
%load_ext autoreload
%autoreload 2

# Filtragem
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#export
from pathlib import Path
import json
import pandas as pd
from datetime import datetime
from anateldb.query import *
from anateldb.constants import console, APP_ANALISE
from unidecode import unidecode
from fastcore.test import *
from fastcore.script import call_parse, Param, store_true, bool_arg
from pyarrow import ArrowInvalid
from rich import print

In [ ]:
#export
def bump_version(version, part=2):
    version = version.split('.')
    version[part] = str(int(version[part]) + 1)
    for i in range(part+1, 3): version[i] = '0'
    return '.'.join(version)

In [ ]:
#export
@call_parse
def formatar_db(
    path: Param("Pasta onde salvar os arquivos", str),
    up_stel: Param("Atualizar a base do Stel", store_true) = False, 
    up_radcom: Param("Atualizar a base do Radcom", store_true) = False,
    up_mosaico: Param("Atualizar a base do Mosaico", store_true) = False,
) -> None:
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    time = datetime.today().strftime("%d/%m/%Y %H:%M:%S")
    console.print(":scroll:[green]Lendo as bases de dados...")
    rd = read_base(path, up_stel, up_radcom, up_mosaico)
    rd['Validade_RF'] = rd.Validade_RF.astype('string').fillna('')
    rd['Data_Ato'] = rd.Data_Ato.astype('string').fillna('')
    rd['Status'] = rd.Status.astype('string')
    rd['Classe'] = rd.Classe.astype('string')
    rd.loc[rd['Status'] != '', 'Status'] = rd.loc[rd['Status'] != '', 'Status'] + ", " \
        + rd.loc[rd['Status'] != '', 'Classe']
    rd.loc[rd['Status'].isna(), 'Status'] = rd.loc[rd['Status'].isna(), 'Num_Serviço'].astype('string')

    rd["Descrição"] = (
        rd.Status
        + ", "
        + rd.Entidade.astype('string').str.title()
        + " ("
        + rd.Fistel.astype('string')
        + ", "
        + rd["Número_da_Estação"].astype('string')
        + "), "
        + rd.Município.astype('string')
        + "/"
        + rd.UF.astype('string')
    )

    export_columns = [
        "Frequência",
        "Latitude",
        "Longitude",
        "Descrição",
        "Num_Serviço",
        "Número_da_Estação",
        "Classe_Emissão",
        "Largura_Emissão",
        "Num_Ato",
        "Data_Ato",
        "Validade_RF",
    ]
    rd = rd.loc[:, export_columns]
    rd.columns = APP_ANALISE
    console.print(":card_file_box:[green]Salvando os arquivos...")
    date = pd.DataFrame(columns=[time])
#    try:
#        rd.to_feather(f"{dest}/AnatelDB.fth")
#    except ArrowInvalid:
#        pass
    with pd.ExcelWriter(f"{dest}/AnatelDB.xlsx") as workbook:
        date.to_excel(workbook, sheet_name="ExtractDate", index=False)
        rd.to_excel(workbook, sheet_name="DataBase", index=False)
    console.print("Sucesso :zap:")
    d = json.loads((dest / 'VersionFile.json').read_text())
    d['anateldb']['ReleaseDate'] = datetime.today().strftime('%d/%m/%Y')
    d['anateldb']['Version'] = bump_version(d['anateldb']['Version']) 
    json.dump(d, (dest / 'VersionFile.json').open('w'))

In [ ]:
pasta = Path(r'G:\Meu Drive\repos\Code\AnatelDatabase')

In [ ]:
if not pasta.exists():
    import subprocess
    result = subprocess.run(['git', 'clone', 'https://github.com/ronaldokun/AnatelDatabase.git'])
    pasta = Path.cwd() / 'AnatelDatabase'

In [ ]:
radcom = read_radcom(pasta) ; radcom.tail()

,Frequência,Fase,Situação,Unidade,Entidade,Fistel,Número da Estação,Município,UF,Latitude,Longitude,CNPJ
4640,87.9,P,M,MHz,"ASSOCIACAO COMUNITARIA DE RADIODIFUSAO, CULTUR...",50414083784,1011019342,Ingazeira,PE,-7.676111,-37.460556,11616328000153
4641,87.9,2,K,MHz,"ASSOCIAÇÃO DE DESENVOLVIMENTO CULTURAL, ARTIST...",50415098327,1011023048,Terra Roxa,PR,-24.370186,-54.013664,06332524000166
4642,87.9,P,M,MHz,ASSOCIACAO CULTURAL COMUNITARIA SHEKINA ...,50416310699,1011031903,Campinas,SP,-22.876112,-47.031387,03065682000109
4643,104.9,1,A,MHz,ASSOCIACAO RADIO COMUNITARIA MONTE SANTO FM,50416345301,1011037472,Monte Santo do Tocantins,TO,-10.005000,-48.988888,19001721000144
4644,104.9,P,M,MHz,ASSOCIACAO RADIO COMUNITARIA TOP FM,50416480004,1011044797,Araguatins,TO,-5.586389,-48.061111,19332116000156


In [ ]:
stel = read_stel(pasta) ; stel.tail()

,Frequência,Unidade,Classe_Emissão,Largura_Emissão,Classe,Num_Serviço,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,CNPJ,Validade_RF
852333,27.910,MHz,J3E,3K00,FX,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695826778,Seropédica,RJ,-22.783333,-43.690556,29427465000105,2021-08-25
852334,27.910,MHz,J3E,3K00,FX,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831291,Seropédica,RJ,-22.791945,-43.681389,29427465000105,2021-08-25
852335,27.910,MHz,J3E,3K00,FX,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831305,Seropédica,RJ,-22.789444,-43.664165,29427465000105,2021-08-25
852336,27.910,MHz,J3E,3K00,FX,19,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831321,Seropédica,RJ,-22.787777,-43.683613,29427465000105,2021-08-25
852337,27.952,MHz,J3E,2K80,FX,19,RAFT TECNOLOGIES BRAZIL SISTEMAS DE TECNOLOGIA...,50411558609,1000352878,Santana de Parnaíba,SP,-23.467583,-46.862110,17196674000106,2035-03-11


In [ ]:
mosaico = read_mosaico(pasta) ; mosaico.tail()

,Serviço,Num_Serviço,Status,Entidade,Fistel,Município,UF,Id,Número_da_Estação,Latitude,Longitude,CNPJ,Validade_RF,Num_Ato,Data_Ato,Frequência,Classe,Canal,Coordenadas_do_Município
23511,RTVD,801,TV-C1,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50439018889,Brasília,DF,60c37be5b1be3,-1,-15.790560,-47.893059,9168704000142,NaN,-1,NaN,479.0,E,15.0,False
23512,RTVD,801,TV-C4,MUNICIPIO DE SAO JOSE DOS CAMPOS,50413209504,São José dos Campos,SP,60c8a035c0f12,-1,-23.153641,-45.907612,46643466000106,2035-10-09,-1,NaN,587.0,C,33.0,False
23513,RTVD,801,TV-C4,TELEVISAO GUAIBA LTDA,50410091049,Santa Maria,RS,60f74a919f757,1004340939,-29.654461,-53.834129,87185468000186,2027-11-30,-1,NaN,503.0,B,19.0,False
23514,RTVD,801,TV-C1,RADIO DIFUSORA DE NOVA RUSSAS LTDA,50439130700,Nova Russas,CE,60f98c8bd3ffa,-1,-4.705800,-40.562099,12364444000195,NaN,-1,NaN,509.0,C,20.0,False
23515,GTVD,247,TV-C4,TELEVISAO GUAIBA LTDA,50405608772,Porto Alegre,RS,6100f691ac50b,691936846,-30.081659,-51.183189,87185468000186,2034-04-19,4942,2019-09-20,515.0,E,21.0,False


In [ ]:
base = read_base(pasta) ; base.head()

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Classe_Emissão,Largura_Emissão
0,0.0280,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,Nova Iguaçu,RJ,-22.662777,-43.476387,2033-08-17,-1,,J9E,8K00
1,0.0285,19,L,OP,COMPANHIA DE GERAÇÃO E TRANSMISSÃO DE ENERGIA ...,50420217282,1494686,Joinville,SC,-26.292500,-48.887222,2025-08-31,-1,,R3E,2K50
2,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,Araporã,MG,-18.410000,-49.099998,2033-08-17,-1,,J3E,1K00
3,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859753,Campinas,SP,-22.774166,-47.004444,2033-08-17,-1,,J3E,1K00
4,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859761,Rio de Janeiro,RJ,-22.926666,-43.264999,2033-08-17,-1,,J3E,500H


In [ ]:
base.tail()

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Classe_Emissão,Largura_Emissão
880494,85468.75,19,L,FX,TIM S A,50417425295,1008754061,Goiânia,GO,-16.592699,-49.267799,2022-08-31,-1,,Q7W,62M5
880495,85468.75,19,L,FX,TIM S A,50417425295,1005059940,Rio Branco,AC,-9.937445,-67.827751,2022-08-31,-1,,Q7W,62M5
880496,85468.75,19,L,FX,TIM S A,50417425295,1009786951,Ananindeua,PA,-1.358931,-48.385670,2022-08-31,-1,,Q7W,750M
880497,85468.75,19,L,FX,TIM S A,50417425295,1009125734,Pará de Minas,MG,-19.857389,-44.616112,2022-08-31,-1,,Q7W,62M5
880498,85468.75,19,L,FX,TIM S A,50417425295,1007183141,São Paulo,SP,-23.624083,-46.623943,2022-08-31,-1,,Q7W,62M5


In [ ]:
formatar_db(pasta)

📜Lendo as bases de dados...

🗃Salvando os arquivos...

Sucesso ⚡

NameError: name 'json' is not defined

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted queries.ipynb.
